In [44]:
import pandas as pd
import numpy as np
import os
from sklearn.model_selection import train_test_split
import statsmodels.api as sm
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, roc_auc_score, classification_report, confusion_matrix
import statsmodels.formula.api as smf
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import StratifiedKFold, cross_validate
from lightgbm import LGBMClassifier
import xgboost as xgb
from catboost import CatBoostClassifier
#import duckdb


os.chdir('/Users/hungduong/Documents/MSBA UofU/MSBA 4th/IS 6813')

In [45]:
finlay=pd.read_csv('final_with_ga_updated.csv')
customer = pd.read_csv("Copy of customer.csv")
#cutoff = pd.read_csv("Copy of cutoff_times.csv")
#ga = pd.read_csv("Copy of google_analytics.csv")
#material  = pd.read_csv("Copy of material.csv")
#hours  = pd.read_csv("Copy of operating_hours.csv")
# orders = pd.read_csv("Copy of orders.csv")
#sales = pd.read_csv("Copy of sales.csv")
#visit =  pd.read_csv("visit_plan.csv")



#ga_w_customer = pd.read_csv('ga_w_customer.csv')
#sales_material_op = pd.read_csv('sales_material_op.csv')
# order_clean = pd.read_csv('order_clean.csv')
# visit_w_cutoff = pd.read_csv('visit_w_cutoff.csv')
#final_table = pd.read_csv('final_table.csv', index_col=0)
#master = pd.read_csv('master.csv', index_col=0)

In [46]:
# Check for missing data here
(finlay.isna().mean() * 100).round(2)

CUSTOMER_ID                   0.00
ANCHOR_DATE                   0.00
NEXT_ANCHOR_SAME_WD           0.00
num_add_to_cart_events        0.00
total_items_added_to_cart     0.00
purchases                     0.00
button_clicks                 0.00
total_events                  0.00
ABANDONED_CART                0.00
SALES_OFFICE                  0.00
SALES_OFFICE_DESC             0.00
DISTRIBUTION_MODE             0.00
SHIPPING_CONDITIONS_DESC      0.00
FREQUENCY_CLEAN              67.24
DISTRIBUTION_MODE_DESC        0.00
WEEK_DAY_OF_ANCHOR_DATE       0.00
WINDOW_FREQUENCY              0.00
CUTOFFTIME__C                 0.00
CREATED_DATE_EST             14.64
CREATED_DATE_UTC             14.64
ORDER_QUANTITY               14.64
ORDER_TYPE                   14.64
MATERIAL_ID                  14.64
ORDER_EXISTS                  0.00
dtype: float64

### 1. Join finlay's window with ga activity (cutoff, visit, ga) with CUSTOMER's chracteristics

In [47]:
customer.head()

,SALES_OFFICE,CUSTOMER_NUMBER,SALES_OFFICE_DESCRIPTION,DISTRIBUTION_MODE_DESCRIPTION,SHIPPING_CONDITIONS_DESCRIPTION,COLD_DRINK_CHANNEL_DESCRIPTION,CUSTOMER_SUB_TRADE_CHANNEL_DESCRIPTION
0,G111,500964906,"Draper, UT",Sideload,48 Hours,Restaurant,Dining
1,G111,501543906,"Draper, UT",Sideload,48 Hours,Restaurant,Mexican Fast Food
2,G293,501394111,"Tacoma, WA",OFS,24 Hours,Attraction,Recreation Center
3,G238,501528559,"Colorado Springs, CO",OFS,48 Hours,Attraction,Recreation Center
4,G132,500504789,"Alamosa, CO",OFS,48 Hours,Restaurant,Fast Food


In [48]:
pd.set_option('display.max_columns', None)
finlay.head()

,CUSTOMER_ID,ANCHOR_DATE,NEXT_ANCHOR_SAME_WD,num_add_to_cart_events,total_items_added_to_cart,purchases,button_clicks,total_events,ABANDONED_CART,SALES_OFFICE,SALES_OFFICE_DESC,DISTRIBUTION_MODE,SHIPPING_CONDITIONS_DESC,FREQUENCY_CLEAN,DISTRIBUTION_MODE_DESC,WEEK_DAY_OF_ANCHOR_DATE,WINDOW_FREQUENCY,CUTOFFTIME__C,CREATED_DATE_EST,CREATED_DATE_UTC,ORDER_QUANTITY,ORDER_TYPE,MATERIAL_ID,ORDER_EXISTS
0,500245685,2025-02-17,2025-02-24,1,37,1,41,261,1,G111,"Draper, UT",OF,48 Hours,NaN,OFS,Monday,7,5:00:00 PM,NaN,NaN,NaN,NaN,NaN,0
1,500245685,2025-03-17,2025-03-24,1,37,1,11,44,1,G111,"Draper, UT",OF,48 Hours,NaN,OFS,Monday,7,5:00:00 PM,NaN,NaN,NaN,NaN,NaN,0
2,500245685,2025-04-14,2025-04-21,1,37,1,7,55,1,G111,"Draper, UT",OF,48 Hours,NaN,OFS,Monday,7,5:00:00 PM,NaN,NaN,NaN,NaN,NaN,0
3,500245685,2025-04-28,2025-05-05,1,37,1,4,38,1,G111,"Draper, UT",OF,48 Hours,NaN,OFS,Monday,7,5:00:00 PM,NaN,NaN,NaN,NaN,NaN,0
4,500245738,2024-11-20,2024-11-27,1,397,1,1,27,0,G111,"Draper, UT",OF,48 Hours,NaN,OFS,Wednesday,7,5:00:00 PM,2024-11-22,2024-11-22 18:46:16+00:00,13.0,MYCOKE360,"[np.float64(129254.0), np.float64(114756.0), n...",1


In [49]:
master_customer = pd.merge(finlay, 
                customer[['CUSTOMER_NUMBER','COLD_DRINK_CHANNEL_DESCRIPTION','CUSTOMER_SUB_TRADE_CHANNEL_DESCRIPTION']],
                left_on='CUSTOMER_ID', 
                right_on='CUSTOMER_NUMBER')

### 2. Get dummy variables for categorical columns and deleted multicollinearity and columns with leakage

In [50]:
data = master_customer.copy()

data['ANCHOR_DATE'] = pd.to_datetime(data['ANCHOR_DATE'], errors='coerce')
data['anchor_date_year'] = data['ANCHOR_DATE'].dt.year
data['anchor_date_month'] = data['ANCHOR_DATE'].dt.month
data['anchor_date_day'] = data['ANCHOR_DATE'].dt.day


data['NEXT_ANCHOR_SAME_WD'] = pd.to_datetime(data['NEXT_ANCHOR_SAME_WD'], errors='coerce')
data['NEXT_ANCHOR_SAME_WD_year'] = data['NEXT_ANCHOR_SAME_WD'].dt.year
data['NEXT_ANCHOR_SAME_WD_year_month'] = data['NEXT_ANCHOR_SAME_WD'].dt.month
data['NEXT_ANCHOR_SAME_WD_year_day'] = data['NEXT_ANCHOR_SAME_WD'].dt.day





data = data.drop(columns=['ANCHOR_DATE','NEXT_ANCHOR_SAME_WD','CREATED_DATE_UTC','CREATED_DATE_EST','MATERIAL_ID','ORDER_QUANTITY','FREQUENCY_CLEAN',
                          'DISTRIBUTION_MODE','ORDER_EXISTS', 'CREATED_DATE_UTC','ORDER_TYPE','ORDER_QUANTITY', 'MATERIAL_ID','purchases','CUSTOMER_NUMBER', ])

data.fillna(0, inplace=True)


In [51]:
data.head()

,CUSTOMER_ID,num_add_to_cart_events,total_items_added_to_cart,button_clicks,total_events,ABANDONED_CART,SALES_OFFICE,SALES_OFFICE_DESC,SHIPPING_CONDITIONS_DESC,DISTRIBUTION_MODE_DESC,WEEK_DAY_OF_ANCHOR_DATE,WINDOW_FREQUENCY,CUTOFFTIME__C,COLD_DRINK_CHANNEL_DESCRIPTION,CUSTOMER_SUB_TRADE_CHANNEL_DESCRIPTION,anchor_date_year,anchor_date_month,anchor_date_day,NEXT_ANCHOR_SAME_WD_year,NEXT_ANCHOR_SAME_WD_year_month,NEXT_ANCHOR_SAME_WD_year_day
0,500245685,1,37,41,261,1,G111,"Draper, UT",48 Hours,OFS,Monday,7,5:00:00 PM,Restaurant,Pizza Fast Food,2025,2,17,2025,2,24
1,500245685,1,37,11,44,1,G111,"Draper, UT",48 Hours,OFS,Monday,7,5:00:00 PM,Restaurant,Pizza Fast Food,2025,3,17,2025,3,24
2,500245685,1,37,7,55,1,G111,"Draper, UT",48 Hours,OFS,Monday,7,5:00:00 PM,Restaurant,Pizza Fast Food,2025,4,14,2025,4,21
3,500245685,1,37,4,38,1,G111,"Draper, UT",48 Hours,OFS,Monday,7,5:00:00 PM,Restaurant,Pizza Fast Food,2025,4,28,2025,5,5
4,500245738,1,397,1,27,0,G111,"Draper, UT",48 Hours,OFS,Wednesday,7,5:00:00 PM,Distributor,Food Distributor,2024,11,20,2024,11,27


In [52]:
data = pd.get_dummies(data, columns=['SALES_OFFICE','SALES_OFFICE_DESC',
                                     'SHIPPING_CONDITIONS_DESC', 'DISTRIBUTION_MODE_DESC', 
                                     'WEEK_DAY_OF_ANCHOR_DATE','WINDOW_FREQUENCY','CUTOFFTIME__C',''
                                     'COLD_DRINK_CHANNEL_DESCRIPTION','CUSTOMER_SUB_TRADE_CHANNEL_DESCRIPTION'], drop_first=True)
pd.set_option('display.max_columns', None) 
print(f'len of data before downsample {len(data)}')
data.head()


len of data before downsample 19492


,CUSTOMER_ID,num_add_to_cart_events,total_items_added_to_cart,button_clicks,total_events,ABANDONED_CART,anchor_date_year,anchor_date_month,anchor_date_day,NEXT_ANCHOR_SAME_WD_year,NEXT_ANCHOR_SAME_WD_year_month,NEXT_ANCHOR_SAME_WD_year_day,SALES_OFFICE_G113,SALES_OFFICE_G115,SALES_OFFICE_G116,SALES_OFFICE_G121,SALES_OFFICE_G131,SALES_OFFICE_G132,SALES_OFFICE_G133,SALES_OFFICE_G141,SALES_OFFICE_G142,SALES_OFFICE_G143,SALES_OFFICE_G144,SALES_OFFICE_G151,SALES_OFFICE_G152,SALES_OFFICE_G161,SALES_OFFICE_G163,SALES_OFFICE_G164,SALES_OFFICE_G171,SALES_OFFICE_G175,SALES_OFFICE_G181,SALES_OFFICE_G182,SALES_OFFICE_G183,SALES_OFFICE_G185,SALES_OFFICE_G221,SALES_OFFICE_G222,SALES_OFFICE_G223,SALES_OFFICE_G235,SALES_OFFICE_G236,SALES_OFFICE_G237,SALES_OFFICE_G238,SALES_OFFICE_G261,SALES_OFFICE_G262,SALES_OFFICE_G263,SALES_OFFICE_G264,SALES_OFFICE_G265,SALES_OFFICE_G266,SALES_OFFICE_G267,SALES_OFFICE_G281,SALES_OFFICE_G282,SALES_OFFICE_G291,SALES_OFFICE_G292,SALES_OFFICE_G293,SALES_OFFICE_G294,"SALES_OFFICE_DESC_Albuquerque, NM","SALES_OFFICE_DESC_Arlington, WA","SALES_OFFICE_DESC_Bellevue, WA","SALES_OFFICE_DESC_Bend, OR","SALES_OFFICE_DESC_Boise, ID","SALES_OFFICE_DESC_Bremerton, WA","SALES_OFFICE_DESC_Cheyenne, WY","SALES_OFFICE_DESC_Colorado Springs, CO","SALES_OFFICE_DESC_Denver, CO","SALES_OFFICE_DESC_Draper, UT","SALES_OFFICE_DESC_Elko, NV","SALES_OFFICE_DESC_Eugene, OR","SALES_OFFICE_DESC_Flagstaff, AZ","SALES_OFFICE_DESC_Glendale, AZ","SALES_OFFICE_DESC_Glenwood Springs, CO","SALES_OFFICE_DESC_Grand Junction, CO","SALES_OFFICE_DESC_Huachuca City, AZ","SALES_OFFICE_DESC_Idaho Falls, ID","SALES_OFFICE_DESC_Johnstown, CO","SALES_OFFICE_DESC_Kingman, AZ","SALES_OFFICE_DESC_LaGrande, OR","SALES_OFFICE_DESC_Lewiston, ID","SALES_OFFICE_DESC_Logan, UT","SALES_OFFICE_DESC_Ogden, UT","SALES_OFFICE_DESC_Pendleton, OR","SALES_OFFICE_DESC_Pocatello, ID","SALES_OFFICE_DESC_Prescott, AZ","SALES_OFFICE_DESC_Price, UT","SALES_OFFICE_DESC_Pueblo, CO","SALES_OFFICE_DESC_Reno, NV","SALES_OFFICE_DESC_Richfield, UT","SALES_OFFICE_DESC_Scottsbluff, NE","SALES_OFFICE_DESC_Show Low, AZ","SALES_OFFICE_DESC_Spokane, WA","SALES_OFFICE_DESC_Tacoma, WA","SALES_OFFICE_DESC_Tempe, AZ","SALES_OFFICE_DESC_Tucson, AZ","SALES_OFFICE_DESC_Twin Falls, ID","SALES_OFFICE_DESC_Walla Walla, WA","SALES_OFFICE_DESC_Wenatchee, WA","SALES_OFFICE_DESC_Wilsonville, OR","SALES_OFFICE_DESC_Yuma, AZ",SHIPPING_CONDITIONS_DESC_48 Hours,SHIPPING_CONDITIONS_DESC_72 Hours,SHIPPING_CONDITIONS_DESC_Dropsite 24 Hours,SHIPPING_CONDITIONS_DESC_Dropsite 48 Hours,DISTRIBUTION_MODE_DESC_E Pallet,DISTRIBUTION_MODE_DESC_Full Service,DISTRIBUTION_MODE_DESC_Night OFS,DISTRIBUTION_MODE_DESC_Night Rapid Delivery,DISTRIBUTION_MODE_DESC_OFS,DISTRIBUTION_MODE_DESC_Rapid Delivery,DISTRIBUTION_MODE_DESC_Sideload,DISTRIBUTION_MODE_DESC_Tell Sell,WEEK_DAY_OF_ANCHOR_DATE_Monday,WEEK_DAY_OF_ANCHOR_DATE_Sunday,WEEK_DAY_OF_ANCHOR_DATE_Thursday,WEEK_DAY_OF_ANCHOR_DATE_Tuesday,WEEK_DAY_OF_ANCHOR_DATE_Wednesday,WINDOW_FREQUENCY_14,WINDOW_FREQUENCY_21,WINDOW_FREQUENCY_28,CUTOFFTIME__C_10:30:00 AM,CUTOFFTIME__C_11:00:00 AM,CUTOFFTIME__C_11:30:00 AM,CUTOFFTIME__C_12:00:00 PM,CUTOFFTIME__C_1:00:00 PM,CUTOFFTIME__C_1:30:00 PM,CUTOFFTIME__C_2:00:00 PM,CUTOFFTIME__C_2:30:00 AM,CUTOFFTIME__C_2:30:00 PM,CUTOFFTIME__C_3:00:00 PM,CUTOFFTIME__C_3:30:00 PM,CUTOFFTIME__C_4:00:00 PM,CUTOFFTIME__C_4:30:00 PM,CUTOFFTIME__C_5:00:00 PM,CUTOFFTIME__C_8:00:00 AM,COLD_DRINK_CHANNEL_DESCRIPTION_Clinic,COLD_DRINK_CHANNEL_DESCRIPTION_Distributor,COLD_DRINK_CHANNEL_DESCRIPTION_Hot Beverage,COLD_DRINK_CHANNEL_DESCRIPTION_Hotel,COLD_DRINK_CHANNEL_DESCRIPTION_Institution,COLD_DRINK_CHANNEL_DESCRIPTION_Restaurant,COLD_DRINK_CHANNEL_DESCRIPTION_Store,COLD_DRINK_CHANNEL_DESCRIPTION_Workplace,CUSTOMER_SUB_TRADE_CHANNEL_DESCRIPTION_Amusement Center,CUSTOMER_SUB_TRADE_CHANNEL_DESCRIPTION_Asian Fast Food,CUSTOMER_SUB_TRADE_CHANNEL_DESCRIPTION_Auto Service,CUSTOMER_SUB_TRADE_CHANNEL_DESCRIPTION_Bar,CUSTOMER_SUB_TRADE_CHANNEL_DESCRIPTION_Beverage Dis

### 3. Downsample, should be 50/50 and shorter rows

In [53]:
# Separate majority and minority
majority = data[data['ABANDONED_CART'] == 0]
minority = data[data['ABANDONED_CART'] == 1]

# Randomly downsample the majority
majority_down = majority.sample(
    n=len(minority),
    random_state=42
)

# Combine
data= pd.concat([majority_down, minority], ignore_index=True)
print(f'len of data after downsample {len(data)}')
print(data['ABANDONED_CART'].value_counts(normalize=True))

len of data after downsample 5706
ABANDONED_CART
0    0.5
1    0.5
Name: proportion, dtype: float64


### 4.Modeling

In [54]:
# Cross validation
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)


In [55]:
# Split data test and train set
x = data.drop(columns=['ABANDONED_CART'])
y = data['ABANDONED_CART']

x_train, x_test, y_train, y_test = train_test_split(
    x, y, test_size=0.3, random_state=42,stratify=y
)


#### simple regression model

In [56]:
# Simple regression model
simple_log_model = LogisticRegression(max_iter=1000, n_jobs=-1, class_weight='balanced')

scores = cross_validate(
    simple_log_model, x, y,
    cv=cv,
    scoring={'roc': 'roc_auc', 'acc': 'accuracy', 'f1': 'f1'},
    return_train_score=False
)

print("ROC-AUC (cv):",  scores['test_roc'].mean().round(3),  "±", scores['test_roc'].std().round(3))
print("Accuracy (cv):", scores['test_acc'].mean().round(3),  "±", scores['test_acc'].std().round(3))
print("F1 (cv):",      scores['test_f1'].mean().round(3),   "±", scores['test_f1'].std().round(3))

ROC-AUC (cv): 0.664 ± 0.013
Accuracy (cv): 0.606 ± 0.003
F1 (cv): 0.651 ± 0.009


#### features importance linearly

In [57]:
# Which columns have strong prediction of abandonment
logit = LogisticRegression(max_iter=1000, class_weight='balanced')
logit.fit(x, y)

# Standardized effect sizes 
stds = x.std(axis=0).replace(0, 1.0)  # guard against zero-variance
std_effect = logit.coef_[0] * stds.values
coef_tbl = (pd.DataFrame({'Feature': x.columns, 'StdEffect': std_effect})
              .assign(Abs=lambda d: d['StdEffect'].abs())
              .sort_values('Abs', ascending=False))
coef_tbl.head(25)

,Feature,StdEffect,Abs
2,total_items_added_to_cart,-5.013868e-01,5.013868e-01
0,CUSTOMER_ID,-7.256637e-02,7.256637e-02
4,total_events,-1.016512e-03,1.016512e-03
3,button_clicks,2.617351e-04,2.617351e-04
5,anchor_date_year,1.576753e-04,1.576753e-04
8,NEXT_ANCHOR_SAME_WD_year,1.536347e-04,1.536347e-04
1,num_add_to_cart_events,-9.755028e-05,9.755028e-05
7,anchor_date_day,4.289460e-05,4.289460e-05
10,NEXT_ANCHOR_SAME_WD_year_day,1.902464e-05,1.902464e-05
6,anchor_date_month,1.311536e-05,1.311536e-05


#### random forest

In [58]:
# Random forest
random_forest = RandomForestClassifier( n_estimators=200,random_state=42,n_jobs=-1, class_weight='balanced')

scores = cross_validate(
    random_forest, x, y,
    cv=cv,
    scoring={'roc': 'roc_auc', 'acc': 'accuracy', 'f1': 'f1'},
    return_train_score=False
)

print("ROC-AUC (cv):",  scores['test_roc'].mean().round(3),  "±", scores['test_roc'].std().round(3))
print("Accuracy (cv):", scores['test_acc'].mean().round(3),  "±", scores['test_acc'].std().round(3))
print("F1 (cv):",      scores['test_f1'].mean().round(3),   "±", scores['test_f1'].std().round(3))

ROC-AUC (cv): 0.774 ± 0.012
Accuracy (cv): 0.701 ± 0.008
F1 (cv): 0.694 ± 0.006


#### xgboost 

In [59]:
# XGBOOST
xgb_model = xgb.XGBClassifier(
    n_estimators=500,
    learning_rate=0.05,
    max_depth=6,
    subsample=0.8,
    colsample_bytree=0.8,
    scale_pos_weight=(y.value_counts()[0] / y.value_counts()[1]),  # handles imbalance
    eval_metric='logloss',
    random_state=42,
    n_jobs=-1
)

scores = cross_validate(
    xgb_model, x, y,
    cv=cv,
    scoring={'roc':'roc_auc', 'acc':'accuracy', 'f1':'f1'},
    return_train_score=False
)

print(pd.DataFrame(scores).mean()[['test_roc', 'test_acc', 'test_f1']])

test_roc    0.772192
test_acc    0.699789
test_f1     0.695583
dtype: float64


#### catboost 

In [60]:
# CatBoost
from catboost import Pool, cv

cat_features = [i for i, c in enumerate(x.columns)
                if str(x[c].dtype) in ('object', 'category')]

train_pool = Pool(x, y, cat_features=cat_features)

params = dict(
    iterations=500,
    learning_rate=0.05,
    depth=6,
    loss_function='Logloss',
    eval_metric='AUC',
    class_weights=[1, (y.value_counts()[0] / y.value_counts()[1])],
    random_seed=42,
    verbose=False
)

cv_results = cv(
    params=params,
    pool=train_pool,
    fold_count=5,
    shuffle=True,
    partition_random_seed=42,
    stratified=True
)

print("ROC-AUC (cv):", round(cv_results['test-AUC-mean'].iloc[-1], 3),
      "±", round(cv_results['test-AUC-std'].iloc[-1], 3))


Training on fold [0/5]

bestTest = 0.7500743771
bestIteration = 491

Training on fold [1/5]

bestTest = 0.7830763616
bestIteration = 389

Training on fold [2/5]

bestTest = 0.7791412736
bestIteration = 471

Training on fold [3/5]

bestTest = 0.782511542
bestIteration = 499

Training on fold [4/5]

bestTest = 0.7532502308
bestIteration = 499

ROC-AUC (cv): 0.769 ± 0.016


#### lightgbm

In [61]:
# 1) Copy and sanitize feature names (keep only letters, numbers, underscores)
X = x.copy()
X.columns = (X.columns
             .str.replace(r'[^0-9A-Za-z_]', '_', regex=True)  # replace bad chars with _
             .str.replace(r'__+', '_', regex=True)            # collapse multiple _
             .str.strip('_'))                                 # trim leading/trailing _

# 1b) Ensure uniqueness after sanitizing (in case two names collapse to same)
def make_unique(cols):
    seen = {}
    out = []
    for c in cols:
        if c in seen:
            seen[c] += 1
            out.append(f"{c}__{seen[c]}")
        else:
            seen[c] = 0
            out.append(c)
    return out

if pd.Index(X.columns).duplicated().any():
    X.columns = make_unique(list(X.columns))

# 2) Cast object columns to 'category' so LightGBM treats them natively
for c in X.columns:
    if X[c].dtype == 'object':
        X[c] = X[c].astype('category')

# 3) Define model and run cross-validate
lgbm = LGBMClassifier(
    n_estimators=600,
    learning_rate=0.05,
    num_leaves=63,
    subsample=0.8,
    colsample_bytree=0.8,
    class_weight='balanced',
    random_state=42
)

cv = StratifiedKFold(5, shuffle=True, random_state=42)
scores = cross_validate(
    lgbm, X, y, cv=cv,
    scoring={'roc':'roc_auc','acc':'accuracy','f1':'f1'},
    return_train_score=False,
    error_score='raise'   # helpful for surfacing future issues
)

import numpy as np
print(pd.DataFrame(scores).mean()[['test_roc','test_acc','test_f1']])

[LightGBM] [Info] Number of positive: 2282, number of negative: 2282
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000574 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1274
[LightGBM] [Info] Number of data points in the train set: 4564, number of used features: 127
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000
[LightGBM] [Info] Number of positive: 2283, number of negative: 2282
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000567 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1271
[LightGBM] [Info] Number of data points in the train set: 4565, number of used features: 128
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.00000